In [1]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Group Assignment
### Team Number: XX
### Team Member Names:
### Team Strategy Chosen: __________(Market Beat, Market Meet, Risk-Free)

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.

In [ ]:
csv_name = "Tickers_Example.csv"
stocks = pd.read_csv(csv_name)

starting_money = 1000000

def isCanadian (ticker):
    return ticker.info['country'] == 'Canada'

# figure out how many shares of a stock you can buy given the price of 
# the stock and some amount of CAD and the current
# exchange rate (1 CAD = X USD)
def shares (ticker, price, money, exchange_rate):
    cad_transaction_fee_per_stock = 0.01/exchange_rate
    cad_max_transaction_fee = 2.15/exchange_rate
    cad_price = price
    
    if not isCanadian(ticker):
        cad_price = price/exchange_rate

    return max(money/(cad_price+cad_transaction_fee_per_stock),
               (money-cad_max_transaction_fee)/cad_price)





# this is for calculating monthly percentage returns of a portfolio of stocks
# also works for just one stock
# this is still under work
def create_df(stocks, dates):
    n = len(stocks)
    start_date = dates[0]
    end_date = dates[len(dates)-1]+pd.Timedelta(days=1)
    
    # creating a dataframe for each ticker with its close price at every first trading day of the month
    stocks_df = []

    
    for ticker in stocks:
        temp_df = ticker.history(start=start_date, end=end_date)  
        temp_df.index = temp_df.index.date
        count = 0

        df_dict = {'Date': dates, 
               'Close': []}

        for i in range (len (temp_df)):
            if temp_df.index[i] == dates[count]:
                close = temp_df['Close'].iloc[i]
                df_dict['Close'].append(close)
                count += 1

        stock_df = pd.DataFrame(df_dict)
        stock_df = stock_df.set_index("Date")
        
        stocks_df.append(stock_df)

    # finding out how many shares in each stock will be bought
    shares_in_stocks = []
    money_in_each_stock = starting_money/n
    
    for i in range (n):
        close = stocks_df[i]['Close'].iloc[0]
        shares_in_stocks.append(money_in_each_stock/close)

    # creating a dataframe that stores the portfolio value at every first trading day of the month
    stock_portfolio = {
        "Date": stocks_df[0].index,
        "Portfolio Value": []}
    
    for i in range (len(stocks_df[0])):
        portfolio_value = 0
        for j in range(n):
            close = stocks_df[j]['Close'].iloc[i]
            portfolio_value += close*shares_in_stocks[j]
            
        stock_portfolio["Portfolio Value"].append(portfolio_value)
    
    stock_portfolio = pd.DataFrame(stock_portfolio)
    stock_portfolio = stock_portfolio.set_index("Date")
    stock_portfolio['Percentage Returns']= stock_portfolio['Portfolio Value'].pct_change()*100
    stock_portfolio.drop(index=stock_portfolio.index[0], inplace = True)
    return stock_portfolio